In [ ]:
import numpy as np
import cv2
import time
import os
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

###원본

In [ ]:
def main():

    input_folder_path = "/content/drive/MyDrive/데이터캡스톤/images/input_yolo"
    output_folder_path = "/content/drive/MyDrive/데이터캡스톤/images/output_images"
    all_images = load_images_from_folder(input_folder_path)
    print('Total Images: ',len(all_images))
    HEIGHT = 32
    WIDTH = 32

    net = cv2.dnn.readNet("/content/drive/MyDrive/데이터캡스톤/yolov4-custom_last.weights", "/content/drive/MyDrive/데이터캡스톤/yolov4-custom.cfg")

    classes = []
    with open("/content/drive/MyDrive/데이터캡스톤/signs_classes_yolo.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    #get last layers names
    layer_names = net.getLayerNames()
    output_layers = []
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    check_time = True
    confidence_threshold = 0.5
    font = cv2.FONT_HERSHEY_SIMPLEX


    detection_confidence = 0.5
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX

    classification_model = load_model('/content/drive/MyDrive/데이터캡스톤/traffic_sign_classifier.h5') #load mask detection model
    classes_classification = []
    with open("/content/drive/MyDrive/데이터캡스톤/signs_classes_cnn.txt", "r") as f:
        classes_classification = [line.strip() for line in f.readlines()]

        results = []
        classification_results = []

        for i in range(len(all_images)):
            img = all_images[i]
            #get image shape
            height, width, channels = img.shape
            #print(img.shape)



            # Detecting objects (YOLO)
            blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            # Showing informations on the screen (YOLO)
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > confidence_threshold:
                        # Object detected
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        # Rectangle coordinates
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]]) + "=" + str(round(confidences[i]*100, 2)) + "%"
                    img = cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 2)
                    crop_img = img[y:y+h, x:x+w]
                    if len(crop_img) >0:
                        crop_img = cv2.resize(crop_img, (WIDTH, HEIGHT))
                        crop_img =  crop_img.reshape(-1, WIDTH,HEIGHT,3)
                        prediction = np.argmax(classification_model.predict(crop_img))
                        print(prediction)
                        label = str(classes_classification[prediction])
                        img = cv2.putText(img, label, (x, y), font, 0.5, (255,0,0), 2)
                        results.append((i, label))

            #cv2.imshow("Image", img)
          #  if cv2.waitKey(1) & 0xFF == ord ('q'):
            #    break
        #cv2.destroyAllWindows()
        results.sort(key=lambda x: x[0])
        for image_index, label in results:

            print("Image:", image_index)
            print("Detected Object:", label)

            output_folder_path = str(output_folder_path)+'/'+ str(i+1) + '.jpg'
            cv2.imwrite(output_folder_path, img)
            output_folder_path = "/content/drive/MyDrive/데이터캡스톤/images/output_images"

main()

Total Images:  505
1/1 [==============================] - 0s 95ms/step
40
1/1 [==============================] - 0s 18ms/step
40
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 18ms/step
12
1/1 [==============================] - 0s 17ms/step
18
1/1 [==============================] - 0s 16ms/step
25
1/1 [==============================] - 0s 18ms/step
21
1/1 [==============================] - 0s 17ms/step
2
1/1 [==============================] - 0s 20ms/step
2
1/1 [==============================] - 0s 18ms/step
40
1/1 [==============================] - 0s 19ms/step
25
1/1 [==============================] - 0s 17ms/step
4
1/1 [==============================] - 0s 22ms/step
9
1/1 [==============================] - 0s 18ms/step
9
1/1 [==============================] - 0s 19ms/step
25
1/1 [==============================] - 0s 19ms/step
23
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 17ms/step
2

In [ ]:
!pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
import ujson as json
import pandas as pd

records = map(json.loads, open('/content/drive/MyDrive/데이터캡스톤/final.ndjson'))
df = pd.DataFrame.from_records(records)

In [ ]:
print(df)

                                            data_row  \
0  {'id': 'clhufa0eof96s07617uifbsnv', 'external_...   
1  {'id': 'clhufa0eof96w0761a8x33tlj', 'external_...   
2  {'id': 'clhufa0eof97007614dwt6si8', 'external_...   
3  {'id': 'clhufa0eof9740761emjpbtno', 'external_...   
4  {'id': 'clhufa0eof9780761csm8bkcr', 'external_...   
5  {'id': 'clhufa0eof97c07610uiufwe1', 'external_...   
6  {'id': 'clhufa0eof97g07610bm3e5ra', 'external_...   
7  {'id': 'clhufa0eof97k07613uyt3x15', 'external_...   
8  {'id': 'clhufa0eof97o07617p9pdn91', 'external_...   
9  {'id': 'clhufa0eof97s0761hsk1fe4o', 'external_...   

                                    media_attributes  \
0  {'height': 800, 'width': 1360, 'mime_type': 'i...   
1  {'height': 800, 'width': 1360, 'mime_type': 'i...   
2  {'height': 800, 'width': 1360, 'mime_type': 'i...   
3  {'height': 800, 'width': 1360, 'mime_type': 'i...   
4  {'height': 213, 'width': 362, 'mime_type': 'im...   
5  {'height': 800, 'width': 1360, 'mime_type': 

In [ ]:
print(df['data_row'].values[0])
print(df['data_row'].values[1])
print(df['data_row'].values[2])
print(df['data_row'].values[3])
print(df['data_row'].values[4])
print(df['data_row'].values[5])
print(df['data_row'].values[6])
print(df['data_row'].values[7])
print(df['data_row'].values[8])
print(df['data_row'].values[9])




{'id': 'clhufa0eof96s07617uifbsnv', 'external_id': '00021.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2F75ddad05-3367-27ea-7005-33bb33e10cec-00021.jpg?Expires=1685342320455&KeyName=labelbox-assets-key-3&Signature=RaeiAtvHXZSMgWNyEPWk_89Qrhc'}
{'id': 'clhufa0eof96w0761a8x33tlj', 'external_id': '00022.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2Fe1c93340-3095-500c-78d2-9a537e3779d0-00022.jpg?Expires=1685342320455&KeyName=labelbox-assets-key-3&Signature=1QNP707gijgYILeYdNyIDeQ_jqY'}
{'id': 'clhufa0eof97007614dwt6si8', 'external_id': '00023.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2Fd2348f49-56ad-34ce-8e60-e98112fae02b-00023.jpg?Expires=1685342320456&KeyName=labelbox-assets-key-3&Signature=IV6et6kBraTfSzvxfwiA-WyDhTo'}
{'id': 'clhufa0eof9740761emjpbtno', 'external_id': '00025.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2F124b2213-934f-06c0-0f9e-cb751bb60b1d-00025.jpg?

In [ ]:
df['external_id'] = df['data_row'].apply(lambda x: x['external_id']) #사진명대로 오름차순 정렬
sorted_df = df.sort_values(by='external_id', ascending=True)
print(sorted_df['data_row'].values[0])
print(sorted_df['data_row'].values[1])
print(sorted_df['data_row'].values[2])
print(sorted_df['data_row'].values[3])
print(sorted_df['data_row'].values[4])

{'id': 'clhufa0eof96s07617uifbsnv', 'external_id': '00021.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2F75ddad05-3367-27ea-7005-33bb33e10cec-00021.jpg?Expires=1685342320455&KeyName=labelbox-assets-key-3&Signature=RaeiAtvHXZSMgWNyEPWk_89Qrhc'}
{'id': 'clhufa0eof96w0761a8x33tlj', 'external_id': '00022.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2Fe1c93340-3095-500c-78d2-9a537e3779d0-00022.jpg?Expires=1685342320455&KeyName=labelbox-assets-key-3&Signature=1QNP707gijgYILeYdNyIDeQ_jqY'}
{'id': 'clhufa0eof97007614dwt6si8', 'external_id': '00023.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2Fd2348f49-56ad-34ce-8e60-e98112fae02b-00023.jpg?Expires=1685342320456&KeyName=labelbox-assets-key-3&Signature=IV6et6kBraTfSzvxfwiA-WyDhTo'}
{'id': 'clhufa0eof9740761emjpbtno', 'external_id': '00025.jpg', 'row_data': 'https://storage.labelbox.com/clhdatsxn1ols073k4dg5040i%2F124b2213-934f-06c0-0f9e-cb751bb60b1d-00025.jpg?

In [ ]:
print(df['projects'].values[0])
print(df['projects'].values[1])
print(df['projects'].values[2])
print(df['projects'].values[3])
print(df['projects'].values[4])
print(df['projects'].values[5])
print(df['projects'].values[6])
print(df['projects'].values[7])
print(df['projects'].values[8])
print(df['projects'].values[9])

{'clhuf49l1036i070t4tiw4zua': {'name': 'small', 'labels': [{'label_kind': 'Default', 'version': '1.0.0', 'id': 'clhufbf0u00v1071a0x6i26na', 'label_details': {'created_at': '2023-05-19T10:36:25.000+00:00', 'updated_at': '2023-05-19T10:36:25.000+00:00', 'created_by': 'firstjiho@khu.ac.kr', 'reviews': []}, 'annotations': {'objects': [], 'classifications': [{'feature_id': 'clhufe0m30002356lqy001n93', 'name': 'yolo', 'checklist_answers': [{'feature_id': 'clhufe0m20001356l1754wr43', 'name': 'Pedestrians', 'classifications': []}]}], 'relationships': []}}]}}
{'clhuf49l1036i070t4tiw4zua': {'name': 'small', 'labels': [{'label_kind': 'Default', 'version': '1.0.0', 'id': 'clhufbq9y043q071w0dsvcxn9', 'label_details': {'created_at': '2023-05-19T10:36:52.000+00:00', 'updated_at': '2023-05-19T10:36:52.000+00:00', 'created_by': 'firstjiho@khu.ac.kr', 'reviews': []}, 'annotations': {'objects': [], 'classifications': [{'feature_id': 'clhufeo6x0004356lt3f7buoe', 'name': 'yolo', 'checklist_answers': [{'fea

In [ ]:
name_value = df['projects'].values[0]['clhuf49l1036i070t4tiw4zua']['name']
print(name_value)


small


In [ ]:
name_value = df['projects'].values[2]['clhuf49l1036i070t4tiw4zua']['labels'][0]['annotations']['classifications'][0]['checklist_answers'][0]['name']
print(name_value)


Speed limit (30km/h)


In [ ]:
external_id = df['data_row'].values[2]['external_id']
print(external_id)


00023.jpg
